In [ ]:
from typing import Optional
from urllib.parse import quote_plus
import pymongo
import certifi
from pymongo.errors import ConnectionFailure
from langchain.tools import tool
import os
from dotenv import load_dotenv
import httpx
import asyncio

# Load environment variables
load_dotenv()

# Safe encoding for MongoDB URI
safe_username = quote_plus(os.getenv("MONGO_USERNAME", ""))
safe_password = quote_plus(os.getenv("MONGO_PASSWORD", ""))
MONGO_URI = (
    f"mongodb+srv://{safe_username}:{safe_password}"
    f"@happmongocluster.mongocluster.cosmos.azure.com/"
    f"?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
)

DB_NAME = os.getenv("DB_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
GEO_KEY = os.getenv("GEOLOCATION_IQ_API_KEY")


async def get_geocode_locationiq(place):
    url = "https://us1.locationiq.com/v1/search.php"
    params = {"key": GEO_KEY, "q": place, "format": "json"}
    async with httpx.AsyncClient() as client:
        res = await client.get(url, params=params)
        data = res.json()
        if data:
            return float(data[0]["lat"]), float(data[0]["lon"])
        return None, None


@tool
async def lookup_provider_info(
    practice_city_name: str,
    primary_taxonomy_description: Optional[str] = None,
    entity_type: str = "Organization",
    npi_number: Optional[int] = None,
    provider_first_name: Optional[str] = None,
    provider_last_name_legal: Optional[str] = None,
    practice_state_name: Optional[str] = None,
    practice_street_address: Optional[str] = None,
    limit: int = 20,
) -> list[dict]:
    """
    Locate healthcare providers or facilities using NPI registry data filtered by location, specialty, or name.

    This tool fetches provider or organization records from a MongoDB collection containing NPPES NPI data.
    It allows filtering by city, state, provider name, taxonomy (specialty), street address, and more.
    The results are further enriched with geographic coordinates (latitude, longitude) using LocationIQ
    for integration with maps or proximity-based queries.

    Args:
        practice_city_name (str): City where the provider or facility is located. (Required)
        primary_taxonomy_description (Optional[str]): Partial match on provider's specialty (e.g., 'cardiology').
        entity_type (str): Either 'Organization' or 'Individual'. Default is 'Organization'.
        npi_number (Optional[int]): Exact NPI number for direct lookup.
        provider_first_name (Optional[str]): First name of the provider (for individuals).
        provider_last_name_legal (Optional[str]): Last name of the provider (for individuals).
        practice_state_name (Optional[str]): State abbreviation (e.g., 'CA', 'NY').
        practice_street_address (Optional[str]): Partial or full street address.
        limit (int): Number of records to return. Default is 20.

    Returns:
        list[dict]: A list of provider/facility records enriched with latitude and longitude, ready for geospatial mapping.
    """

    
    query = {
        "practice_city_name": {"$regex": f"^{practice_city_name.strip()}$", "$options": "i"},
        "entity_type": entity_type
    }

    if npi_number:
        query["npi_number"] = npi_number

    if provider_first_name:
        query["provider_first_name"] = {"$regex": f"^{provider_first_name.strip()}$", "$options": "i"}

    if provider_last_name_legal:
        query["provider_last_name_legal"] = {"$regex": f"^{provider_last_name_legal.strip()}$", "$options": "i"}

    if practice_state_name:
        query["practice_state_name"] = practice_state_name.strip().upper()

    if practice_street_address:
        query["practice_street_address"] = {"$regex": f".*{practice_street_address.strip()}.*", "$options": "i"}

    if primary_taxonomy_description:
        query["primary_taxonomy_description"] = {"$regex": f".*{primary_taxonomy_description.strip()}.*", "$options": "i"}

    try:
        client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]
        print("✅ Connected to MongoDB.")

        raw_results = list(collection.find(query, {"_id": 0}).limit(limit))
        print(f"🔍 Found {len(raw_results)} result(s).")

        enriched_results = []
        for rec in raw_results:
            address = f"{rec.get('practice_street_address', '')}, {rec.get('practice_city_name', '')}, {rec.get('practice_state_name', '')}"
            lat, lon = await get_geocode_locationiq(address)
            rec["latitude"] = lat
            rec["longitude"] = lon
            enriched_results.append(rec)

        return enriched_results

    except ConnectionFailure as conn_fail:
        print(f"❌ MongoDB connection failed. {conn_fail}")
        return []
    except Exception as e:
        print(f"❌ Error: {e}")
        return []
    finally:
        if client:
            client.close()
            print("🔌 MongoDB connection closed.")

In [2]:
print(lookup_provider_info.description)

Fetch provider details from MongoDB based on filters, with fuzzy search and lat/lon enrichment.

Required:
    - practice_city_name (str): City where the provider practices.

Optional:
    - primary_taxonomy_description (str): Fuzzy match against human-readable taxonomy.
    - entity_type (str): Default is 'Organization'.
    - npi_number, provider_first_name, provider_last_name_legal, practice_state_name
    - practice_street_address (partial match allowed)
    - limit: Number of results to return

Returns:
    List of dicts with enriched latitude/longitude.


In [3]:
print(lookup_provider_info.args)

{'practice_city_name': {'title': 'Practice City Name', 'type': 'string'}, 'primary_taxonomy_description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Primary Taxonomy Description'}, 'entity_type': {'default': 'Organization', 'title': 'Entity Type', 'type': 'string'}, 'npi_number': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Npi Number'}, 'provider_first_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Provider First Name'}, 'provider_last_name_legal': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Provider Last Name Legal'}, 'practice_state_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Practice State Name'}, 'practice_street_address': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Practice Street Address'}, 'limit': {'default': 20, 'title': 'Limit', 'type': 'integer'}}


In [4]:
query = {
    "practice_city_name": "San Antonio",
    "practice_street_address": "Santa Rosa"
}

response = await lookup_provider_info.ainvoke(query)

C:\Users\Bapan Bairagya\AppData\Local\Temp\ipykernel_16084\2390851473.py:93: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
d:\API Developement\multi_agents_health_wellness_app\.venv\Lib\site-packages\pymongo\synchronous\mongo_client.py:1737: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


✅ Connected to MongoDB.
🔍 Found 13 result(s).
🔌 MongoDB connection closed.


In [18]:
response

[{'npi_number': 1770768525,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'TEJANO EMERGENCY PHYSICIANS PLLC',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'SAN ANTONIO',
  'practice_state_name': 'TX',
  'practice_postal_code': 782073108,
  'practice_street_address': '333 N SANTA ROSA AVE',
  'practice_phone_number': '2107042880',
  'taxonomy_codes_list': ['207P00000X'],
  'primary_taxonomy_code': '207P00000X',
  'taxonomy_descriptions_list': ['Emergency Medicine'],
  'primary_taxonomy_description': 'Emergency Medicine',
  'latitude': 29.427025,
  'longitude': -98.498978},
 {'npi_number': 1023293875,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'TEXAS EM-1 MEDICAL SERVICES, PA',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'SAN ANTONIO',
  'practice_state_name': 'TX',
  'practice_postal_code': 782073108,
  'practice_street_address': '333 N SANTA ROSA AVE',
  'practice

In [19]:
query = {
    "practice_city_name": "San Antonio",
    "practice_street_address": "Santa Rosa",
    "primary_taxonomy_description": "emergency"
}

response1 = await lookup_provider_info.ainvoke(query)

C:\Users\Bapan Bairagya\AppData\Local\Temp\ipykernel_19600\2390851473.py:93: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
d:\API Developement\multi_agents_health_wellness_app\.venv\Lib\site-packages\pymongo\synchronous\mongo_client.py:1737: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


✅ Connected to MongoDB.
🔍 Found 2 result(s).
🔌 MongoDB connection closed.


In [20]:
response1

[{'npi_number': 1770768525,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'TEJANO EMERGENCY PHYSICIANS PLLC',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'SAN ANTONIO',
  'practice_state_name': 'TX',
  'practice_postal_code': 782073108,
  'practice_street_address': '333 N SANTA ROSA AVE',
  'practice_phone_number': '2107042880',
  'taxonomy_codes_list': ['207P00000X'],
  'primary_taxonomy_code': '207P00000X',
  'taxonomy_descriptions_list': ['Emergency Medicine'],
  'primary_taxonomy_description': 'Emergency Medicine',
  'latitude': 29.427025,
  'longitude': -98.498978},
 {'npi_number': 1104001957,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'TRAVIS EMERGENCY PHYSICIANS PLLC',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'SAN ANTONIO',
  'practice_state_name': 'TX',
  'practice_postal_code': 782073108,
  'practice_street_address': '333 N SANTA ROSA AVE',
  'practic

In [21]:
query = {
    
    "practice_city_name": "Houston",
    "practice_street_address": "Gessner",
    "primary_taxonomy_description": "emergency"
}

response2 = await lookup_provider_info.ainvoke(query)

C:\Users\Bapan Bairagya\AppData\Local\Temp\ipykernel_19600\2390851473.py:93: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
d:\API Developement\multi_agents_health_wellness_app\.venv\Lib\site-packages\pymongo\synchronous\mongo_client.py:1737: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


✅ Connected to MongoDB.
🔍 Found 4 result(s).
🔌 MongoDB connection closed.


In [22]:
response2

[{'npi_number': 1629252473,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'TRANSCARE, LLC',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'HOUSTON',
  'practice_state_name': 'TX',
  'practice_postal_code': 770635139,
  'practice_street_address': '3300 S GESSNER RD STE 208',
  'practice_phone_number': '7132668996',
  'taxonomy_codes_list': ['343900000X'],
  'primary_taxonomy_code': '343900000X',
  'taxonomy_descriptions_list': ['Non-emergency Medical Transport (VAN)'],
  'primary_taxonomy_description': 'Non-emergency Medical Transport (VAN)',
  'latitude': 29.728584,
  'longitude': -95.539178},
 {'npi_number': 1730460569,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'MED-PORT MEDICAL TRANSPORTATION SERVICES',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'HOUSTON',
  'practice_state_name': 'TX',
  'practice_postal_code': 770742915,
  'practice_street_address': '8787 S G

In [23]:
query = {
    
    "practice_city_name": "Houston",
    "practice_street_address": "Gessner",
    "primary_taxonomy_description": "Pediatrics"
}

response3 = await lookup_provider_info.ainvoke(query)

C:\Users\Bapan Bairagya\AppData\Local\Temp\ipykernel_19600\2390851473.py:93: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
d:\API Developement\multi_agents_health_wellness_app\.venv\Lib\site-packages\pymongo\synchronous\mongo_client.py:1737: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


✅ Connected to MongoDB.
🔍 Found 6 result(s).
🔌 MongoDB connection closed.


In [24]:
response3

[{'npi_number': 1538489356,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'JOYCE EGBE, MD. PA.',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'HOUSTON',
  'practice_state_name': 'TX',
  'practice_postal_code': 770712803,
  'practice_street_address': '12609 S GESSNER DR',
  'practice_phone_number': '7137732700',
  'taxonomy_codes_list': ['208000000X'],
  'primary_taxonomy_code': '208000000X',
  'taxonomy_descriptions_list': ['Pediatrics'],
  'primary_taxonomy_description': 'Pediatrics',
  'latitude': 29.6379315,
  'longitude': -95.5307342},
 {'npi_number': 1922400043,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'LA PROVIDENCE PEDIATRIC AND FAMILY CLINICS',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'HOUSTON',
  'practice_state_name': 'TX',
  'practice_postal_code': 770711008,
  'practice_street_address': '9900 S GESSNER RD',
  'practice_phone_number': '7139816002',

In [5]:
query = {
    
    'practice_city_name': "Fort Worth",
    "primary_taxonomy_description": "Neurology"
}

response4 = await lookup_provider_info.ainvoke(query)

C:\Users\Bapan Bairagya\AppData\Local\Temp\ipykernel_16084\2390851473.py:93: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
d:\API Developement\multi_agents_health_wellness_app\.venv\Lib\site-packages\pymongo\synchronous\mongo_client.py:1737: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


✅ Connected to MongoDB.
🔍 Found 20 result(s).
🔌 MongoDB connection closed.


In [6]:
response4

[{'npi_number': 1851870711,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'MERCY PSYCHIATRY INC',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'FORT WORTH',
  'practice_state_name': 'TX',
  'practice_postal_code': 762444480,
  'practice_street_address': '4941 GOLDEN TRIANGLE BLVD STE 911',
  'practice_phone_number': '8177793716',
  'taxonomy_codes_list': ['2084P0805X'],
  'primary_taxonomy_code': '2084P0805X',
  'taxonomy_descriptions_list': ['Psychiatry & Neurology - Geriatric Psychiatry'],
  'primary_taxonomy_description': 'Psychiatry & Neurology - Geriatric Psychiatry',
  'latitude': 32.9313795,
  'longitude': -97.2778235},
 {'npi_number': 1033699343,
  'entity_type': 'Organization',
  'provider_org_name_legal': 'MENS-SANA TELEPSYCHIATRY PLLC',
  'provider_last_name_legal': nan,
  'provider_first_name': nan,
  'practice_city_name': 'FORT WORTH',
  'practice_state_name': 'TX',
  'practice_postal_code': 761232120,
  'practice